In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from itertools import product
from datetime import date
from path import Path
import pandas as pd
import random

import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'

from mlbt.run_bt import run_bt
from mlbt.feature_eng import define_feature_configs
from mlbt.load_data import LENGTH_RANKING


DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
s_csv = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

# Code to execute the pipeline inside a multiprocessing.Pool
# This creates the payloads that "run_frontend_payloads.ipynb" can use to generate the
# frontend payloads (.json loaded by the frontend)
ranking = ["equity_index", ""]


symbol_groups = list(
    {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }.keys()
)

features = define_feature_configs()
print(len(features))
c_s = (len(features) // 32) + 1 
c_s = 1
print('c_s', c_s)
data_sim = {
    "start_date": [date(2000, 1, 1)],
    "end_date": [date(2021, 1, 1)],
    "symbol_groups": [symbol_groups],
#     "symbols": [['@NQ#C']],
#     "symbols": [[x] for x in LENGTH_RANKING],
    "bar_type": ["dollar"],
    "binarize": ["fixed_horizon"],
#     "binarize_params": [6, 12],
#     "binarize_params": [6],
    "binarize_params": [6, 12, 25, 50],
#     "binarize_params": [25, 50, 100],
#     "binarize_params": [12],
    "alpha": ["none"],
#     "alpha": ["ma-cross_100_1000"],
#     "features": [features[i:i+c_s] for i in range(0, len(features), c_s)],
    "features": [features],
    "classifier": ["random_forest", "xgboost", "lgbm", "dummy"],
#     "classifier": ["lgbm"],
#     "classifier": ["knn"],
#     "classifier": ["xgboost", "lgbm", "dummy"],
#     "classifier": ["random_forest", "xgboost", "dummy"],
#     "classifier": ["tpot"],
#     "classifier": ["lgbm"],
    "num_threads": [32], # how many parallel hyperopti 
    "n_jobs": [32], # how many parallel infer
    "num_threads": [1], # how many parallel hyperopti 
    "n_jobs": [1], # how many parallel infer
#     "feat_imp_n_estimators": [1000],
#     "feat_imp_cv": [10],
    "func": [run_bt],
#     "feature_calc_only": [True],
#     "feature_imp_only": [True],
#     "check_completed": [True],
#     "optimize_hypers": [False],
#     "reuse_hypers": [False],
#     "load_from_disk": [False],
#     "skip_feature_imp": [True],
#     "pca_transform": [False],
#     "standard_scale": [False],
#     "hypers_n_iter": [25],
}

33
c_s 1


In [12]:
datas = list(dict(zip(data_sim.keys(), x)) for x in product(*data_sim.values()))
datas
# for data in datas:
#     data['features'] = list(sorted(data['features'], key=lambda x:random.random()))


[{'start_date': datetime.date(2000, 1, 1),
  'end_date': datetime.date(2021, 1, 1),
  'symbol_groups': ['agriculture',
   'currency',
   'energy',
   'equity_index',
   'interest_rate',
   'metals'],
  'bar_type': 'dollar',
  'binarize': 'fixed_horizon',
  'binarize_params': 6,
  'alpha': 'none',
  'features': [{'name': 'log_ret'},
   {'name': 'close', 'symbol': 'VIX.XO'},
   {'name': 'ffd', 'd': 0.5},
   {'name': 'roll', 'window': 10},
   {'name': 'rollimp', 'window': 10},
   {'name': 'amihud', 'window': 10},
   {'name': 'kyle', 'window': 10},
   {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 25},
   {'name': 'rollimp', 'window': 25},
   {'name': 'amihud', 'window': 25},
   {'name': 'kyle', 'window': 25},
   {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 50},
   {'name': 'rollimp', 'window': 50},
   {'name': 'amihud', 'window': 50},
   {'name': 'kyle', 'window': 50},
   {'name': 'ewm_

In [13]:
len(datas)

16

In [14]:
# [x['features'][0] for x in datas]

In [15]:
from mlbt.multiprocess import process_jobs

In [16]:
num_threads = max(1, min(32, len(datas)) - 0)
num_threads
      

16

In [18]:
res = process_jobs(datas, num_threads=num_threads)

2020-02-21 01:09:13.361370 100.0% run_bt done after 10.18 minutes. Remaining 0.0 minutes.


In [9]:
for data in datas:
    run_bt(**data)

2020-02-21 00:58:04,612 config: {'DATA_DIR': Path('C:\\Users\\doda/Dropbox/algotrading/data'), 'F_PAYLOAD_DIR': Path('C:\\Users\\doda/pr/fincl/frontend/public/payloads'), 'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'data_freq': 'minutely', 'vol_estimate': 100, 'downsampling': 'cusum', 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'features': [{'name': 'log_ret'}, {'name': 'close', 'symbol': 'VIX.XO'}, {'name': 'ffd', 'd': 0.5}, {'name': 'roll', 'window': 10}, {'name': 'rollimp', 'window': 10}, {'name': 'amihud', 'window': 10}, {'name': 'kyle', 'window': 10}, {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 25}, {'name': 'rollimp', 'window': 25}, {'name': 'amihud', 'window': 25}, {'name': 'kyle', 'window': 25}, {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 50}, {'name': 'rollimp', 'window': 50}, {'name': '

2020-02-21 00:58:12,265 We have the payload, not recomputing
2020-02-21 00:58:12,311 config: {'DATA_DIR': Path('C:\\Users\\doda/Dropbox/algotrading/data'), 'F_PAYLOAD_DIR': Path('C:\\Users\\doda/pr/fincl/frontend/public/payloads'), 'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'data_freq': 'minutely', 'vol_estimate': 100, 'downsampling': 'cusum', 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'features': [{'name': 'log_ret'}, {'name': 'close', 'symbol': 'VIX.XO'}, {'name': 'ffd', 'd': 0.5}, {'name': 'roll', 'window': 10}, {'name': 'rollimp', 'window': 10}, {'name': 'amihud', 'window': 10}, {'name': 'kyle', 'window': 10}, {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 25}, {'name': 'rollimp', 'window': 25}, {'name': 'amihud', 'window': 25}, {'name': 'kyle', 'window': 25}, {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}}, {'name': 'roll',

2020-02-21 00:58:19,910 We have the payload, not recomputing
2020-02-21 00:58:19,957 config: {'DATA_DIR': Path('C:\\Users\\doda/Dropbox/algotrading/data'), 'F_PAYLOAD_DIR': Path('C:\\Users\\doda/pr/fincl/frontend/public/payloads'), 'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'data_freq': 'minutely', 'vol_estimate': 100, 'downsampling': 'cusum', 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'features': [{'name': 'log_ret'}, {'name': 'close', 'symbol': 'VIX.XO'}, {'name': 'ffd', 'd': 0.5}, {'name': 'roll', 'window': 10}, {'name': 'rollimp', 'window': 10}, {'name': 'amihud', 'window': 10}, {'name': 'kyle', 'window': 10}, {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 25}, {'name': 'rollimp', 'window': 25}, {'name': 'amihud', 'window': 25}, {'name': 'kyle', 'window': 25}, {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}}, {'name': 'roll',

2020-02-21 00:58:27,592 corrupted payload: C:\Users\doda/Dropbox/algotrading/data\payloads\payload_agriculture-currency-energy-equity_index-interest_rate-metals_dollar_fixed_horizon_25_none_[]_dummy.json
2020-02-21 00:58:27,604 Symbols: ['@AD#C', '@BP#C', '@CD#C', '@ED#C', '@ES#C', '@NQ#C', '@GF#C', '@LE#C', '@HE#C', '@EMD#C', '@NKD#C', '@NE#C', '@RP#C', 'BD#C', 'EZ#C', 'XG#C', 'EX#C', '@YM#C', '@BO#C', '@C#C', '@FV#C', '@TY#C', '@TU#C', '@UB#C', '@O#C', '@SM#C', '@S#C', '@W#C', '@MME#C', '@CC#C', '@CT#C', '@OJ#C', '@KC#C', '@SB#C', 'GAS#C', 'LF#C', 'LG#C', 'QCL#C', 'QHO#C', 'QNG#C', 'QPA#C', 'QPL#C', '@QG#C', '@QM#C', 'QGC#C', 'QHG#C', 'QSI#C']
2020-02-21 00:58:27,652 @AD#C: Have 84868 bars and 401 binarized events
2020-02-21 00:58:27,664 @BP#C: Have 92410 bars and 5393 binarized events
2020-02-21 00:58:27,676 @CD#C: Have 86716 bars and 2397 binarized events
2020-02-21 00:58:27,686 @ED#C: Have 48965 bars and 14296 binarized events
2020-02-21 00:58:27,697 @ES#C: Have 58387 bars and 274

2020-02-21 00:58:36,764 @OJ#C: Feature engineering for 33 features
2020-02-21 00:58:37,040 Joined 33 features into (72862, 33) shape
2020-02-21 00:58:37,040 @KC#C: Feature engineering for 33 features
2020-02-21 00:58:37,325 Joined 33 features into (86403, 33) shape
2020-02-21 00:58:37,325 @SB#C: Feature engineering for 33 features
2020-02-21 00:58:37,608 Joined 33 features into (87741, 33) shape
2020-02-21 00:58:37,609 GAS#C: Feature engineering for 33 features
2020-02-21 00:58:37,873 Joined 33 features into (27979, 33) shape
2020-02-21 00:58:37,874 LF#C: Feature engineering for 33 features
2020-02-21 00:58:38,147 Joined 33 features into (63915, 33) shape
2020-02-21 00:58:38,148 LG#C: Feature engineering for 33 features
2020-02-21 00:58:38,306 could not load_hdf
Traceback (most recent call last):
  File "C:\Users\doda\pr\fincl\backend\mlbt\load_data.py", line 176, in load_hdf
    return pd.read_hdf(path, 'table')
  File "c:\users\doda\anaconda3\envs\fincl\lib\site-packages\pandas\io\py

Traceback (most recent call last):
  File "c:\users\doda\anaconda3\envs\fincl\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-ddb706853d84>", line 2, in <module>
    run_bt(**data)
  File "C:\Users\doda\pr\fincl\backend\mlbt\run_bt.py", line 461, in run_bt
    deck = run_feature_engineering(config, deck)
  File "C:\Users\doda\pr\fincl\backend\mlbt\feature_eng.py", line 259, in run_feature_engineering
    feat = engineer_feature(deck, symbol, config, feat_config)["Close"]
  File "C:\Users\doda\pr\fincl\backend\mlbt\feature_eng.py", line 312, in engineer_feature
    feat = compute_feature(deck, for_symbol, config, feat_conf, symbol, df)
  File "C:\Users\doda\pr\fincl\backend\mlbt\feature_eng.py", line 324, in compute_feature
    feat = FEATURES[feat_name](df, **params)
  File "C:\Users\doda\pr\fincl\backend\mlbt\feature_eng.py", line 94, in ffd
    return frac_diff_ffd(np.log(df[col

KeyboardInterrupt: 

In [ ]:
%debug